#**Inventory Management System for High-Demand Products**

This notebook presents an inventory management system tailored for high-demand products identified through a demand forecasting model. Effective inventory management is critical for optimizing stock levels, reducing holding costs, and ensuring that customer demand is met without overstocking.

###**Objective**
The primary goal of this system is to calculate and manage inventory metrics for high-demand products, specifically focusing on:

**Optimal Order Quantity:** The ideal amount of stock to order at once.

**Buffer Stock:** The safety stock level maintained to account for uncertainties in demand or supply.

**Reorder Trigger:** The inventory level at which a new order should be placed.
**Current Stock Level:** The actual amount of stock currently available.

**Place Order Decision:** A yes/no decision on whether an order should be placed based on current stock levels and reorder triggers.

This system is designed to streamline the inventory management process by providing data-driven recommendations, reducing the risk of stockouts or excess inventory, and ultimately improving operational efficiency.

#Importing Libraries and Loading Data

In [1]:
import math
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import joblib

# Load your dataset from a CSV file
df = pd.read_csv('synthetic_walmart_data.csv')

# Select the relevant columns using .loc
df = df.loc[:, ['Product ID', 'Product Category', 'Store ID', 'Annual Demand',
                'Ordering Cost', 'Holding Cost', 'Lead Time',
                'Standard Deviation of Demand', 'Current Stock Level']]

# Assuming high-demand products are provided from the demand forecasting model
high_demand_products = [9, 23]  # Replace with actual product IDs from your forecasting model

# Filter the DataFrame to include only high-demand products
high_demand_df = df.loc[df['Product ID'].isin(high_demand_products)]


FileNotFoundError: [Errno 2] No such file or directory: 'synthetic_walmart_data.csv'

#Creating the Machine Learning Model

In [2]:
# Features and target variable for the model
X = high_demand_df[['Annual Demand', 'Ordering Cost', 'Holding Cost', 'Lead Time', 'Standard Deviation of Demand']]
y = high_demand_df['Current Stock Level']

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train a simple linear regression model
model = LinearRegression()
model.fit(X_train, y_train)

# Test the model
predictions = model.predict(X_test)

# Evaluate the model (you can add more metrics here if needed)
model_score = model.score(X_test, y_test)
print(f"Model R^2 score: {model_score:.2f}")


NameError: name 'high_demand_df' is not defined

#Saving and Loading the Model

In [3]:
# Save the model to a file using joblib
joblib.dump(model, 'inventory_management_model.joblib')

# Later, you can load the model using this code
# loaded_model = joblib.load('inventory_management_model.joblib')


NameError: name 'model' is not defined

#Calculating Inventory Metrics

In [4]:
# Function to calculate inventory metrics using the trained model
def calculate_inventory_metrics(product_id, model):
    # Filter the DataFrame for the given product ID
    product_row = df.loc[df['Product ID'] == product_id]

    if product_row.empty:
        print(f"Product ID {product_id} not found in the dataset.")
        return None

    # Prepare input features for prediction
    X_new = product_row[['Annual Demand', 'Ordering Cost', 'Holding Cost', 'Lead Time', 'Standard Deviation of Demand']]

    # Predict the current stock level
    predicted_stock_level = model.predict(X_new)[0]

    # Calculate optimal order quantity (EOQ)
    optimal_order_quantity = math.sqrt(2 * product_row['Annual Demand'].values[0] * product_row['Ordering Cost'].values[0] / product_row['Holding Cost'].values[0])

    # Calculate average daily demand
    average_daily_demand = product_row['Annual Demand'].values[0] / 365

    # Calculate buffer stock (Safety Stock) assuming a Z-value of 1.65 for a 95% service level
    buffer_stock = 1.65 * product_row['Standard Deviation of Demand'].values[0] * math.sqrt(product_row['Lead Time'].values[0])

    # Calculate reorder trigger (Reorder Point)
    reorder_trigger = average_daily_demand * product_row['Lead Time'].values[0] + buffer_stock

    # Determine if an order should be placed
    place_order = 'Yes' if predicted_stock_level <= reorder_trigger else 'No'

    return {
        'Product ID': product_id,
        'Optimal Order Quantity': int(optimal_order_quantity),
        'Buffer Stock': int(buffer_stock),
        'Reorder Trigger': int(reorder_trigger),
        'Current Stock Level': int(predicted_stock_level),
        'Place Order?': place_order
    }




#Using the Loaded Model

In [5]:
# If you need to load and use the model later
loaded_model = joblib.load('inventory_management_model.joblib')

# Example usage with the loaded model
product_metrics = calculate_inventory_metrics(9, loaded_model)



FileNotFoundError: [Errno 2] No such file or directory: 'inventory_management_model.joblib'

In [6]:
product_metrics

NameError: name 'product_metrics' is not defined